# HW3: Netflix Data Analysis

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import scipy.sparse
import random

In [2]:
ratings = scipy.sparse.load_npz('sample_matrix.npz')

In [3]:
ratings_test = scipy.sparse.load_npz('sample_matrix_test.npz')

In [4]:
test_value = np.load('test_value.npz')['arr_0']
test_movie = np.load('test_movie.npz')['arr_0']
test_user = np.load('test_user.npz')['arr_0']
train_value = np.load('train_value.npz')['arr_0']
train_movie = np.load('train_movie.npz')['arr_0']
train_user = np.load('train_user.npz')['arr_0']

In [19]:
# Get features
movie_avg = np.zeros(5000)
movie_revs = np.zeros(5000)
# movie_pos = np.zeros(5000)
# movie_neg = np.zeros(5000)
user_avg = np.zeros (10000)
user_revs = np.zeros(10000)
# user_pos = np.zeros(10000)
# user_neg = np.zeros(10000)

for i in range(5000):
    movie_slice = ratings[i, :]
    movie_avg[i] = csr_matrix.mean(movie_slice)
    movie_revs[i] = csr_matrix.count_nonzero(movie_slice)
#     count_neg = 0
#     count_pos = 0
#     for j in range(movie_slice.shape[1]):
#         if movie_slice[0,j] > 4:
#             count_pos+=1
#         elif (movie_slice[0,j] < 2 and movie_slice[0,j] > 0):
#             count_neg+=1
#     movie_pos[i] = count_pos
#     movie_neg[i] = count_neg
    
for i in range(10000):
    user_slice = ratings[:, i]
    user_avg[i] = csr_matrix.mean(user_slice)
    user_revs[i] = csr_matrix.count_nonzero(user_slice)
#     count_neg = 0
#     count_pos = 0
#     for j in range(user_slice.shape[1]):
#         if user_slice[0,j] > 4:
#             count_pos+=1
#         elif (user_slice[0,j] < 2 and user_slice[0,j] > 0):
#             count_neg+=1
#     user_pos[i] = count_pos
#     user_neg[i] = count_neg

In [20]:
#Movie avg, user avg, # movie reviews, # user reviews, # positive movie reviews, # negative movie review
# #positive user reiews, # negative user reviews
movie_avg_train = np.zeros(len(train_value))
user_avg_train = np.zeros(len(train_value))
movie_revs_train = np.zeros(len(train_value))
user_revs_train = np.zeros(len(train_value))
# movie_pos_train = np.zeros(len(train_value))
# user_pos_train = np.zeros(len(train_value))
# movie_neg_train = np.zeros(len(train_value))
# user_neg_train = np.zeros(len(train_value))

for i in range(len(train_value)):
    movie_avg_train[i] = movie_avg[train_movie[i]]
    user_avg_train[i] = user_avg[train_user[i]]
    movie_revs_train[i] = movie_revs[train_movie[i]]
    user_revs_train[i] = user_revs[train_user[i]]
#     movie_pos_train[i] = movie_pos[train_moive[i]]
#     user_pos_train[i] = user_pos[train_user[i]]
#     movie_neg_train[i] = movie_neg[train_movie[i]]
#     user_neg_train[i] = user_neg[train_movie[i]]

# features_train = np.vstack((movie_avg_train, user_avg_train, movie_revs_train, user_revs_train,
#                             movie_pos_train, user_pos_train, movie_neg_train, user_neg_train)).T

features_train = np.vstack((movie_avg_train, user_avg_train, movie_revs_train, user_revs_train)).T


movie_avg_test = np.zeros(len(test_value))
user_avg_test = np.zeros(len(test_value))
movie_revs_test = np.zeros(len(test_value))
user_revs_test = np.zeros(len(test_value))
user_revs_test = np.zeros(len(test_value))
# movie_pos_test = np.zeros(len(test_value))
# user_pos_test = np.zeros(len(test_value))
# movie_neg_test = np.zeros(len(test_value))
# user_neg_test = np.zeros(len(test_value))

for i in range(len(test_value)):
    movie_avg_test[i] = movie_avg[test_movie[i]]
    user_avg_test[i] = user_avg[test_user[i]]
    movie_revs_test[i] = movie_revs[test_movie[i]]
    user_revs_test[i] = user_revs[test_user[i]]
#     movie_pos_test[i] = movie_pos[test_moive[i]]
#     user_pos_test[i] = user_pos[test_user[i]]
#     movie_neg_test[i] = movie_neg[test_movie[i]]
#     user_neg_test[i] = user_neg[test_movie[i]]

# features_test = np.vstack((movie_avg_test, user_avg_test, movie_revs_test, user_revs_test,
#                            movie_pos_test, user_pos_test, movie_neg_test, user_neg_test)).T
features_test = np.vstack((movie_avg_test, user_avg_test, movie_revs_test, user_revs_test)).T

In [21]:
def MSE(predictions, test_set):
    summation = 0
    for i in range(len(predictions)):
        summation += (predictions[i] - test_set[i])**2
    return summation/len(predictions)

In [22]:
def MSE_non_zero(predictions, test_set):
    summation = 0
    non_zero = 0
    for i in range(len(predictions)):
        value = test_set[i]
        if value > 0:
            summation += (predictions[i] - value)**2
            non_zero += 1
    return summation/non_zero

In [23]:
print MSE(movie_avg_test, test_value)
print MSE(user_avg_test, test_value)
print MSE_non_zero(movie_avg_test, test_value)
print MSE_non_zero(user_avg_test, test_value)

0.0295266520192
0.0292378472977
13.4983016894
13.2433637058


In [24]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(features_train, train_value)
predictions = lr.predict(features_test)

/usr/local/lib/python2.7/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [25]:
print lr.score(features_test, test_value)

0.0469320084328


In [26]:
print MSE(predictions, test_value)
print MSE_non_zero(predictions, test_value)

0.0286764894139
12.7348214851


In [27]:
from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(features_train, train_value)
predictions = ridge.predict(features_test)

In [28]:
print ridge.score(features_test, test_value)

0.0469322475302


In [29]:
print MSE(predictions, test_value)
print MSE_non_zero(predictions, test_value)

0.0286764822202
12.7348664267


In [30]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(features_train, train_value)
predictions = lasso.predict(features_test)

In [31]:
print MSE(predictions, test_value)
print MSE_non_zero(predictions, test_value)

0.0295029533853
13.7036123355


In [32]:
print lasso.score(features_test, test_value)

0.0194643381485


In [33]:
from sklearn.decomposition import TruncatedSVD

In [40]:
# This file consists of titles and release years associated with each ID
movie_titles = pd.read_csv('movie_titles.txt', header = None, names = ['ID','Year','Name'])
print(movie_titles.head())

   ID    Year                          Name
0   1  2003.0               Dinosaur Planet
1   2  2004.0    Isle of Man TT 2004 Review
2   3  1997.0                     Character
3   4  1994.0  Paula Abdul's Get Up & Dance
4   5  2004.0      The Rise and Fall of ECW


In [42]:
n_components = 100
svd = TruncatedSVD(n_components = n_components)

In [43]:
Z = svd.fit_transform(ratings_test)

In [44]:
components = svd.components_

In [46]:
print Z.shape

(5000, 100)


In [47]:
from sklearn.cluster import KMeans
clusters = 20
kmeans = KMeans(init='k-means++', n_init=10, n_clusters=clusters)
kmeans.fit(Z)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=20, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [ ]:
labels = kmeans.labels_
similar_users_avg = np.zeros(5000)

clusters = 

for i in range(len(test_value)):
    label = labels[test_movie[i]]
    